In [22]:
import pandas as pd
import sys
import os
import numpy as np
import yfinance as yf
import torch.nn as nn
from sklearn.model_selection import train_test_split

import sys
import os
sys.path.append(os.path.abspath("../src"))
from data_prep_utils import *

In [23]:
context_tickers = {
    '30_year_bonds': 'TLT',
    '10_year_bonds': 'IEF',
    '5_year_bonds': 'IEI',
    '2_year_bonds': 'SHY',
    '1_year_bonds': 'SHV',
    'gold': 'GLD',
    'silver': 'SLV',
    # 'copper': 'CPER', # starts in 2011
    'oil': 'USO',
    'natural_gas': 'UNG',
    'sp500': 'SPY',
    'nasdaq': 'QQQ',
    'dow_jones': 'DIA',
    'russell_2000': 'IWM',
    'us_dollar': 'UUP',
    'emerging_markets': 'EEM',
    'euro': 'FXE',
    'british_pound': 'FXB',
    'japanese_yen': 'FXY',
    # 'bitcoin': 'BTC-USD', # starts in 2014
    # 'ethereum': 'ETH-USD', # start in 2014
}

target_tickers = {
    'costco': 'COST',
    # 'coinbase': 'COIN',
    # 'robinhood': 'HOOD',
    # 'amazon': 'AMZN',
    # 'apple': 'AAPL',
    # 'google': 'GOOGL',
    # 'microsoft': 'MSFT',
    # 'tesla': 'TSLA',
    # 'meta': 'META',
    # 'nvidia': 'NVDA',
    # 'general_motors': 'GM',
    # 'ford': 'F',
    # 'crowdstrike': 'CRWD',
    # 'palantir': 'PLTR',
}

START_DATE = '2010-01-01'
END_DATE = '2025-06-20'

In [27]:
# Download and enrich data
context_data = download_and_enrich_data(context_tickers, START_DATE, END_DATE)    
context_df = pd.concat(context_data.values(), axis=1).dropna()
target_data = download_and_enrich_data(target_tickers, START_DATE, END_DATE)    
target_df = pd.concat(target_data.values(), axis=1).dropna()
combined_df = pd.concat([context_df, target_df], axis=1).dropna()

# verify timeframes
# for ticker, df in context_data.items():
#     print(f"{ticker} index range: {df.index.min()} to {df.index.max()}", 'count:', len(df))

context_df.to_csv('../data/context_data.csv')
combined_df.to_csv('../data/training_raw.csv')

print('Num days in context_df:', len(context_df))
print('Num days in target_df:', len(target_df))
print('Num days in combined_df:', len(combined_df))
print(context_df.shape)
display(context_df.tail())


c:\Users\saris\OneDrive\Desktop\active inf investing\src\data_prep_utils.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start, end=end)
[*********************100%***********************]  1 of 1 completed
c:\Users\saris\OneDrive\Desktop\active inf investing\src\data_prep_utils.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start, end=end)
[*********************100%***********************]  1 of 1 completed
c:\Users\saris\OneDrive\Desktop\active inf investing\src\data_prep_utils.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start, end=end)
[*********************100%***********************]  1 of 1 completed
c:\Users\saris\OneDrive\Desktop\active inf investing\src\data_prep_utils.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.down

Num days in context_df: 3870
Num days in target_df: 3870
Num days in combined_df: 3870
(3870, 234)


,TLT_Close,TLT_High,TLT_Low,TLT_Open,TLT_Volume,TLT_EMA_12,TLT_EMA_26,TLT_MACD,TLT_Signal,TLT_BB_upper,...,FXY_Open,FXY_Volume,FXY_EMA_12,FXY_EMA_26,FXY_MACD,FXY_Signal,FXY_BB_upper,FXY_BB_lower,FXY_OBV,FXY_ATR
Date,,,,,,,,,,,,,,,,,,,,,
2025-06-12,87.169998,87.199997,86.599998,86.889999,43017800,85.930698,86.027221,-0.096523,-0.351946,87.170142,...,64.209999,219600,63.896924,63.825327,0.071597,0.102388,64.687385,63.104614,30394100.0,0.498571
2025-06-13,86.330002,86.879997,85.779999,86.730003,49374000,85.992129,86.049649,-0.057520,-0.293061,87.237552,...,63.810001,138700,63.915858,63.839747,0.076111,0.097133,64.673140,63.190860,30255400.0,0.471429
2025-06-16,85.459999,86.430000,85.459999,86.080002,36496600,85.910263,86.005971,-0.095708,-0.253590,87.197879,...,64.029999,173500,63.864188,63.820507,0.043682,0.086443,64.614042,63.295958,30081900.0,0.448571
2025-06-17,86.500000,86.660004,85.750000,86.000000,34521600,86.000992,86.042566,-0.041574,-0.211187,87.292689,...,63.630001,128600,63.800467,63.793062,0.007405,0.070635,64.628695,63.262304,29953300.0,0.446429
2025-06-18,86.650002,87.089996,86.349998,86.879997,38589000,86.100839,86.087561,0.013278,-0.166294,87.425877,...,63.599998,218500,63.745010,63.766909,-0.021899,0.052128,64.645143,63.215857,29734800.0,0.447143


In [26]:

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, Dataset

# Define horizons for target predictions (trading days)
horizons = {"1_day": 1, "1_week": 5, "1_month": 21, "1_year": 252}

# Step 1: Create Target Columns
def create_targets(df, target_column, horizons):
    """
    Generate shifted target columns for specified horizons.
    """
    for horizon, shift in horizons.items():
        df[f"Target_{horizon}"] = df[target_column].shift(-shift)
    return df

# Step 2: Normalize Features
def normalize_features(df, exclude_columns):
    """
    Normalize input features using Min-Max scaling, excluding specified columns.
    """
    scaler = MinMaxScaler()
    input_features = df.drop(columns=exclude_columns)
    scaled_features = scaler.fit_transform(input_features)
    
    # Reconstruct normalized DataFrame
    normalized_df = pd.DataFrame(scaled_features, columns=input_features.columns, index=input_features.index)
    for column in exclude_columns:
        normalized_df[column] = df[column]  # Add back excluded columns (e.g., targets)
    
    return normalized_df, scaler

# Step 3: Create Sequences for Transformer Input
def create_sequences(data, target_columns, seq_length):
    """
    Create sequences for transformer input with specified sequence length.
    """
    X, y = [], []
    data_values = data.drop(columns=target_columns).values
    target_values = data[target_columns].values

    for i in range(len(data) - seq_length):
        X.append(data_values[i:i + seq_length])
        y.append(target_values[i + seq_length])
    
    return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

# Step 4: Custom Dataset for PyTorch
class StockDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Full Data Preparation Pipeline
def prepare_data_pipeline(df, target_column, sequence_length=30):
    """
    Prepare data for transformer input: normalize, create targets, and generate sequences.
    """
    # Create targets
    df = create_targets(df, target_column, horizons)

    # Normalize features
    exclude_columns = [f"Target_{horizon}" for horizon in horizons.keys()]
    df, scaler = normalize_features(df, exclude_columns)

    # Drop NaN values
    df.dropna(inplace=True)

    # Split data into training, validation, and test sets
    train_df, test_df = train_test_split(df, test_size=0.2, shuffle=False)
    train_df, val_df = train_test_split(train_df, test_size=0.2, shuffle=False)

    # Create sequences
    target_columns = exclude_columns
    train_X, train_y = create_sequences(train_df, target_columns, sequence_length)
    val_X, val_y = create_sequences(val_df, target_columns, sequence_length)
    test_X, test_y = create_sequences(test_df, target_columns, sequence_length)

    # Create DataLoaders
    batch_size = 32
    train_loader = DataLoader(StockDataset(train_X, train_y), batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(StockDataset(val_X, val_y), batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(StockDataset(test_X, test_y), batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader, scaler

# Example Usage
# Assuming combined_df is the enriched DataFrame
target_column = target_tickers['costco'] + "_Close"
sequence_length = 256 # 21 = 1 month, 252 = 1 year

# Prepare the data
train_loader, val_loader, test_loader, scaler = prepare_data_pipeline(combined_df, target_column, sequence_length)

# Display the shapes of the datasets
print(f"Train Loader: {len(train_loader.dataset)} samples")     
print(f"Validation Loader: {len(val_loader.dataset)} samples")
print(f"Test Loader: {len(test_loader.dataset)} samples")


Train Loader: 1736 samples
Validation Loader: 243 samples
Test Loader: 367 samples
